<a href="https://colab.research.google.com/github/OVP2023/NN/blob/main/%D0%94%D0%97_Inception_v3_%D0%90%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D1%83%D1%80%D1%8B_%D1%81%D0%B2%D0%B5%D1%80%D1%82%D0%BE%D1%87%D0%BD%D1%8B%D1%85_%D1%81%D0%B5%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary
import time
import numpy as np

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 10 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. "
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

In [ ]:
transoforms = tv.transforms.Compose([tv.transforms.Grayscale(3), tv.transforms.Resize((224, 224)), tv.transforms.ToTensor()])
Batch_size = 64
train_dataset = tv.datasets.EMNIST('.', split='balanced', train=True, download=True, transform=transoforms)
test_dataset = tv.datasets.EMNIST('.', split='balanced', train=False, download=True, transform=transoforms)

train_data = torch.utils.data.DataLoader(train_dataset, batch_size=Batch_size, shuffle=True)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=Batch_size)

100%|██████████| 562M/562M [00:02<00:00, 213MB/s]


In [ ]:
train_dataset

Dataset EMNIST
    Number of datapoints: 112800
    Root location: .
    Split: Train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=3)
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

In [ ]:
print(len(train_dataset.classes))


47


In [ ]:
model_inception_v3 = tv.models.inception_v3(pretrained=False, aux_logits=False)
model_inception_v3.fc = nn.Linear(in_features=2048, out_features=47)
optimizer_inception_v3 = torch.optim.Adam(model_inception_v3.parameters(), lr=0.001)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [ ]:
summary(model_inception_v3.to(device), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 111, 111]             864
       BatchNorm2d-2         [-1, 32, 111, 111]              64
       BasicConv2d-3         [-1, 32, 111, 111]               0
            Conv2d-4         [-1, 32, 109, 109]           9,216
       BatchNorm2d-5         [-1, 32, 109, 109]              64
       BasicConv2d-6         [-1, 32, 109, 109]               0
            Conv2d-7         [-1, 64, 109, 109]          18,432
       BatchNorm2d-8         [-1, 64, 109, 109]             128
       BasicConv2d-9         [-1, 64, 109, 109]               0
        MaxPool2d-10           [-1, 64, 54, 54]               0
           Conv2d-11           [-1, 80, 54, 54]           5,120
      BatchNorm2d-12           [-1, 80, 54, 54]             160
      BasicConv2d-13           [-1, 80, 54, 54]               0
           Conv2d-14          [-1, 192,

In [12]:

train(model_inception_v3, train_data, test_data, optimizer_inception_v3,1)

Step 0. time since epoch: 39.128. Train acc: 0.031. Train Loss: 3.890
Step 10. time since epoch: 376.465. Train acc: 0.047. Train Loss: 3.918
Step 20. time since epoch: 711.502. Train acc: 0.087. Train Loss: 3.707
Step 30. time since epoch: 1036.757. Train acc: 0.137. Train Loss: 3.413
Step 40. time since epoch: 1360.823. Train acc: 0.202. Train Loss: 3.102
Step 50. time since epoch: 1686.416. Train acc: 0.259. Train Loss: 2.833
Step 60. time since epoch: 2009.037. Train acc: 0.311. Train Loss: 2.597
Step 70. time since epoch: 2358.159. Train acc: 0.359. Train Loss: 2.392
Step 80. time since epoch: 2676.917. Train acc: 0.393. Train Loss: 2.238
Step 90. time since epoch: 2995.082. Train acc: 0.428. Train Loss: 2.100
Step 100. time since epoch: 3316.810. Train acc: 0.457. Train Loss: 1.976
Step 110. time since epoch: 3637.916. Train acc: 0.482. Train Loss: 1.875
Step 120. time since epoch: 3959.086. Train acc: 0.505. Train Loss: 1.782
Step 130. time since epoch: 4285.663. Train acc: 0.52

KeyboardInterrupt: 